In [1]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

col_names = ['temperature','occurrence_of_nausea','lumbar_pain','urine_pushing','micturition_pains','burning_of_urethra','decision_1','decision_2']
dataset_inflammation = pd.read_table('AcuteInflammations.data', header=None, names=col_names, sep=',')
print("\nDataset Acute Inflammations:")
dataset_inflammation

/home/larissa/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: FutureWarning: read_table is deprecated, use read_csv instead.
  


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [ ]:
dataset_inflammation.columns

In [ ]:
inflammation_values = dataset_inflammation.iloc[:,0:8].values
print("\nAcute Inflammations features:\n")
print(inflammation_values)

In [ ]:
inflammation_normalizada = dataset_inflammation.values.copy()
normalizador =  StandardScaler()

inflammation_normalizada = normalizador.fit_transform(dataset_inflammation)
dataset_inflammation['temperature'] = inflammation_normalizada[:,0]

print("\nDataset Acute Inflammation normalizada:")
dataset_inflammation

In [ ]:
inflammation_classes = dataset_inflammation.iloc[:,8].values
print("\nAcute Inflammations classes:\n")
print(inflammation_classes)
print("\nAcute Inflammations classes shape:")
print(inflammation_classes.shape)

In [ ]:
kf = model_selection.StratifiedKFold(n_splits = 2)
arvore_decisao = tree.DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)
vizinhos_proximos = KNeighborsClassifier(n_neighbors=3, weights = 'distance')
naive_bayes_gaussian = GaussianNB()
regressao_logistica = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='saga', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
rede_neural = MLPClassifier(hidden_layer_sizes=(5,), activation="logistic", max_iter= 300, alpha=0.001, solver="sgd", tol=1e-4, verbose=True, learning_rate_init=.01)

In [ ]:
predicted_classes = dict()
predicted_classes['arvore_decisao'] = np.zeros(inflammation_classes.shape)
predicted_classes['vizinhos_proximos'] = np.zeros(inflammation_classes.shape)
predicted_classes['naive_bayes_gaussian'] = np.zeros(inflammation_classes.shape)
predicted_classes['regressao_logistica'] = np.zeros(inflammation_classes.shape)
predicted_classes['rede_neural'] = np.zeros(inflammation_classes.shape)

In [ ]:

for train, test in kf.split(inflammation_values, inflammation_classes):
    data_train, target_train = inflammation_values[train], inflammation_classes[train]
    data_test, target_test = inflammation_values[test], inflammation_classes[test]

    arvore_decisao = arvore_decisao.fit(data_train, target_train)
    arvore_decisao_predicted = arvore_decisao.predict(data_test)
    predicted_classes['arvore_decisao'][test] = arvore_decisao_predicted

    vizinhos_proximos = vizinhos_proximos.fit(data_train, target_train)
    vizinhos_proximos_predicted = vizinhos_proximos.predict(data_test)
    predicted_classes['vizinhos_proximos'][test] = vizinhos_proximos_predicted

    naive_bayes_gaussian = naive_bayes_gaussian.fit(data_train, target_train)
    naive_bayes_gaussian_predicted = naive_bayes_gaussian.predict(data_test)
    predicted_classes['naive_bayes_gaussian'][test] = naive_bayes_gaussian_predicted

    regressao_logistica = regressao_logistica.fit(data_train, target_train)
    regressao_logistica_predicted = regressao_logistica.predict(data_test)
    predicted_classes['regressao_logistica'][test] = regressao_logistica_predicted

    rede_neural = rede_neural.fit(data_train, target_train)
    rede_neural_predicted = rede_neural.predict(data_test)
    predicted_classes['rede_neural'][test] = rede_neural_predicted

In [ ]:
for classificador in predicted_classes.keys():
    print("================================================================================================")
    print("Resultados do classificador %s\n%s\n"
          %(classificador, metrics.classification_report(inflammation_classes, predicted_classes[classificador])))
    print("Matriz de confusão: \n%s\n\n\n" % metrics.confusion_matrix(inflammation_classes, predicted_classes[classificador]))
